In [2]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import numpy as np

# Enable inline plotting
%matplotlib inline
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)  

Python version 3.6.0 |Anaconda custom (64-bit)| (default, Dec 23 2016, 13:19:00) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
Pandas version 0.21.1
Matplotlib version 2.0.2


In [53]:
Location = r'train.csv'
df = pd.read_csv(Location, parse_dates=['booking_date', 'dep_date'])
#pd.to_datetime(pd.Series(sLength, index=df.index))
#print(type(df["Hour"][0]))

#testing Data
Location = r'test.csv'
test_df = pd.read_csv(Location, parse_dates=['booking_date', 'dep_date'])
#print(type(test_df["Hour"][0]))

sLength  = len(df['pax'])
#df['dep_time'] = pd.to_datetime(pd.Series(sLength, index=df.index))
#test_df['Day'] = pd.Series(sLength, index=df.index)

originList = df.origin.unique()
df = df.assign(id=(df['origin'] + '_' + df['destination']).astype('category').cat.codes)
df['days'] = ((df['dep_date'] - df['booking_date'])/np.timedelta64(1, 'D')).astype(int)/365
df['fare_choice_sc'] = df.groupby(['fare_choice']).ngroup()
df['dep_time_sc'] = ((df['dep_time']%100).astype(int)/5 + (df['dep_time']/100).astype(int)*12)/(24*12)
#df['id2'] = df.groupby(['destination']).ngroup()
maxID  = max(df['id'])
df['id'] = df['id']/maxID
print(maxID)

test_df = test_df.assign(id=(test_df['origin'] + '_' + test_df['destination']).astype('category').cat.codes)
test_df['days'] = ((test_df['dep_date'] - test_df['booking_date'])/np.timedelta64(1, 'D')).astype(int)/365
test_df['dep_time_sc'] = ((test_df['dep_time']%100).astype(int)/5 + (test_df['dep_time']/100).astype(int)*12)/(24*12)
#df['id2'] = df.groupby(['destination']).ngroup()
maxID  = max(test_df['id'])
test_df['id'] = test_df['id']/maxID
print(maxID)

df.head(5)
test_df.head()

#print(df.origin.unique())
#print(test_df.origin.unique())
#print(df.destination.unique())
#print(test_df.destination.unique())
#df.destination.unique()

14
14


,id,booking_date,origin,destination,dep_date,dep_time,pax,days,dep_time_sc
0,0.071429,2017-04-21,BLR,DEL,2017-06-24,1745,2,0.175342,0.739583
1,0.285714,2017-05-24,BOM,DEL,2017-06-24,1700,2,0.084932,0.708333
2,0.071429,2017-04-12,BLR,DEL,2017-06-24,700,1,0.200000,0.291667
3,0.071429,2017-06-21,BLR,DEL,2017-06-24,1125,2,0.008219,0.475694
4,0.214286,2017-05-31,BOM,CCU,2017-06-24,1905,1,0.065753,0.795139


In [58]:
#select columns here for features
flightDataPd = df[['id','pax', 'days','dep_time_sc','fare_choice_sc']]
flightData = flightDataPd.as_matrix(columns=flightDataPd.columns[0:])
print(flightData)
#change the length of the features based on the selection
featureLength = 4
X = flightData[:,0:featureLength]
#print(X)
y = flightData[:,featureLength]
#print(data_y_train)
df.head()

[[ 1.          2.          0.30410959  0.27083333  1.        ]
 [ 0.57142857  1.          0.01369863  0.73958333  0.        ]
 [ 0.28571429  1.          0.00547945  0.375       3.        ]
 ..., 
 [ 0.57142857  3.          0.10410959  0.40625     1.        ]
 [ 0.42857143  1.          0.06575342  0.58680556  1.        ]
 [ 0.35714286  1.          0.00821918  0.35763889  0.        ]]


,id,booking_date,origin,destination,dep_date,dep_time,pax,fare_choice,days,fare_choice_sc,dep_time_sc
0,1.000000,2017-02-10,MAA,DEL,2017-06-01,630,2,Deal,0.304110,1,0.270833
1,0.571429,2017-05-27,DEL,BOM,2017-06-01,1745,1,Classic,0.013699,0,0.739583
2,0.285714,2017-05-30,BOM,DEL,2017-06-01,900,1,Saver,0.005479,3,0.375000
3,0.142857,2017-05-30,BOM,BLR,2017-06-01,2130,2,Classic,0.005479,0,0.895833
4,0.571429,2017-05-30,DEL,BOM,2017-06-01,1240,1,Classic,0.005479,0,0.527778


In [54]:
#select columns here for features
flightDataTestPd = test_df[['id','pax', 'days','dep_time_sc']]
flightDataTest = flightDataTestPd.as_matrix(columns=flightDataTestPd.columns[0:])

#change the length of the features based on the selection
data_X_testF = flightDataTest[:,0:featureLength]
#data_y_testF = flightDataTest[:,featureLength-1]
#print(data_y_test)
test_df.head()

,id,booking_date,origin,destination,dep_date,dep_time,pax,days,dep_time_sc
0,0.071429,2017-04-21,BLR,DEL,2017-06-24,1745,2,0.175342,0.739583
1,0.285714,2017-05-24,BOM,DEL,2017-06-24,1700,2,0.084932,0.708333
2,0.071429,2017-04-12,BLR,DEL,2017-06-24,700,1,0.200000,0.291667
3,0.071429,2017-06-21,BLR,DEL,2017-06-24,1125,2,0.008219,0.475694
4,0.214286,2017-05-31,BOM,CCU,2017-06-24,1905,1,0.065753,0.795139


In [65]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

data_X_train, data_X_test, data_y_train, data_y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Create linear regression object
regr = linear_model.LogisticRegression()

# Train the model using the training sets
regr.fit(data_X_train, data_y_train)


# predict class labels for the test set
#predicted = regr.predict(data_X_test)
predicted = regr.predict_proba(data_X_test)
print(predicted)

df_y_pred = pd.DataFrame(predicted)
print(df_y_pred.head(20))
prediction_location = r'prediction.csv'
df_y_pred.to_csv(prediction_location, index=False, header=['Classic', 'Deal', 'Flex', 'Saver'])

# The mean squared error
#print("Mean absolute error: %.2f"
#      % mean_absolute_error(data_y_test, predicted))
# Explained variance score: 1 is perfect prediction
#print('Variance score: %.2f' % r2_score(data_y_test, predicted))


#polynomial_features = PolynomialFeatures(degree=3, include_bias=False)

#pipeline = Pipeline([("polynomial_features", polynomial_features),("linear_regression", regr)])
#pipeline.fit(data_X_train, data_y_train)

    # Evaluate the models using crossvalidation
#scores = cross_val_score(pipeline, data_X_train, data_y_train, scoring="neg_mean_squared_error", cv=10)
#print(scores)

#pred_deg = pipeline.predict(data_X_test)
#print(pred_deg)
#print(data_y_test)

# The mean squared error
#print("Mean absolute error: %.2f"
#      % mean_absolute_error(data_y_test, pred_deg))
# Explained variance score: 1 is perfect prediction
#print('Variance score: %.2f' % r2_score(data_y_test, pred_deg))

#print(len(pred_deg))
#print(len(data_y_test))
#print(metrics.accuracy_score(data_y_test, pred_deg))
#print(metrics.confusion_matrix(data_y_test, pred_deg))

[[  5.79831498e-01   8.17737077e-02   1.04589576e-01   2.33805219e-01]
 [  6.27724641e-01   5.55370425e-02   1.42703957e-01   1.74034360e-01]
 [  5.45038309e-01   1.43785528e-01   5.81068091e-02   2.53069353e-01]
 ..., 
 [  6.25814186e-01   7.86771116e-02   7.70791391e-02   2.18429564e-01]
 [  5.26488769e-02   6.32050862e-01   1.24433966e-02   3.02856865e-01]
 [  1.70422812e-05   7.80191880e-01   3.58511471e-05   2.19755226e-01]]
           0         1         2         3
0   0.579831  0.081774  0.104590  0.233805
1   0.627725  0.055537  0.142704  0.174034
2   0.545038  0.143786  0.058107  0.253069
3   0.442326  0.174584  0.063351  0.319739
4   0.306135  0.319755  0.051969  0.322142
5   0.371745  0.294280  0.049423  0.284551
6   0.624263  0.080843  0.114298  0.180596
7   0.578331  0.063948  0.092389  0.265333
8   0.000818  0.752639  0.000414  0.246129
9   0.630496  0.043458  0.129652  0.196394
10  0.092582  0.659234  0.018485  0.229698
11  0.247707  0.429395  0.069784  0.253115
12  0.5

### Final test for submission

In [154]:
pipeline.fit(X, y)

# Make predictions using the testing set
data_y_pred = pipeline.predict(data_X_testF)
df_y_pred = pd.DataFrame(data_y_pred)
prediction_location = r'prediction.csv'
df_y_pred.to_csv(prediction_location, index=False, header=['AvgWtTime'])
prediction_location = r'train_mod.csv'
np.savetxt("train_mod.csv", flightData, delimiter=",")
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
#print("Mean squared error: %.2f"
#      % mean_squared_error(data_y_test, data_y_pred))
# Explained variance score: 1 is perfect prediction
#print('Variance score: %.2f' % r2_score(data_y_test, data_y_pred))

Coefficients: 
 [ -7.64530259e+00  -1.29502594e+00   2.14665581e-01  -2.91826937e+01
  -3.46828210e-01   2.22305555e+00  -4.83827052e-02  -5.44089625e-03
   1.66356177e+00   1.73914026e-02   9.69630244e-02  -1.32595301e-02
   1.72320568e+00   1.93442589e-01   3.84552808e-04  -1.44564484e-01
  -9.20029442e-03   1.33276125e+01   1.34698060e+00  -3.23458632e-02
  -1.83319994e-01  -1.83174282e-02  -6.44107336e-04   7.57756678e-02
   8.46564313e-03   9.36540718e-03   2.36708747e-04  -5.08547557e-02
  -6.61870677e-03  -8.42549411e-06   3.60231759e-03   1.67912415e-04
  -3.72282061e-01  -4.59669943e-02   2.93164663e-03  -1.93596081e-03
   1.96755893e-04  -2.25508813e-02  -8.22450170e-03  -1.14503013e-05
   3.97763056e-03   4.20073522e-04  -3.35132874e-01  -6.80967000e-02
   2.18342613e-03   1.14505587e-07  -7.29166959e-05  -4.13667626e-06
   1.53668232e-02   1.60319105e-03   3.90086178e-05  -1.06926744e+00
  -1.53161182e-01  -2.15356466e-03  -6.59889272e-04]


In [ ]:
print(data_X_test.shape)
#print(data_y_test.shape)

In [117]:
index = 138
df['Day'][index] = int(df['Date'][index].day)
print(type(df['Day'][index]))

print(type(df['Date'][index].dayofweek))

<class 'numpy.int64'>
<class 'int'>


/Users/eabraham/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [64]:
flightDataTestPd.head(20)

,Day,Hour,Total,Flights,Booths
0,1,0,300,2,12
1,1,4,143,1,4
2,1,6,411,2,13
3,1,7,312,1,16
4,1,8,427,2,14
5,1,9,238,1,15
6,1,10,885,5,17
7,1,11,556,3,17
8,1,12,708,4,25
9,1,13,2469,11,33
